In [1]:
# -*- coding: utf-8 -*-
import json, time, random, ast
import numpy as np
import requests
from requests.adapters import HTTPAdapter
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from dataclasses import dataclass
from typing import List, Tuple

BASE_URL = "https://limittheory.aictf.sg:5000"

# ============ Shared parts (rate-limit + backoff) ============
RATE_MIN_INTERVAL = 0.7
LAST_CALL_TS = 0.0
def pace():
    global LAST_CALL_TS
    now = time.monotonic()
    wait = LAST_CALL_TS + RATE_MIN_INTERVAL - now
    if wait > 0:
        time.sleep(wait)
    LAST_CALL_TS = time.monotonic()

def make_session():
    s = requests.Session()
    s.mount("https://", HTTPAdapter(max_retries=0))
    s.headers.update({"Content-Type": "application/json"})
    return s
session = make_session()

def request_with_backoff(method: str, path: str, payload=None, timeout=10, max_attempts=12):
    url = f"{BASE_URL}{path}"
    backoff = 1.0
    for _ in range(max_attempts):
        pace()
        try:
            resp = session.get(url, timeout=timeout) if method == "GET" else session.post(url, data=json.dumps(payload or {}), timeout=timeout)
        except requests.RequestException:
            time.sleep(backoff + random.uniform(0,0.3))
            backoff = min(backoff*1.7, 12.0)
            continue
        if resp.status_code == 429:
            ra = resp.headers.get("Retry-After")
            try:
                wait = float(ra) if ra else backoff
            except:
                wait = backoff
            time.sleep(min(max(wait,0.5),15.0) + random.uniform(0,0.4))
            backoff = min(backoff*1.8, 20.0)
            continue
        if 500 <= resp.status_code < 600:
            time.sleep(backoff + random.uniform(0,0.5))
            backoff = min(backoff*1.7, 12.0)
            continue
        resp.raise_for_status()
        try:
            return resp.json()
        except:
            return {"raw": resp.text}
    raise RuntimeError("Exceeded attempts")

def api_get(path, timeout=10):
    return request_with_backoff("GET", path, None, timeout)

def api_post(path, payload, timeout=10):
    return request_with_backoff("POST", path, payload, timeout)

# ============ Experiment + model ============
def experiment_eval(c, e, s, p):
    payload = {
        "coconut_milk": float(np.clip(c, 0, 10)),
        "eggs": float(np.clip(e, 0, 10)),
        "sugar": float(np.clip(s, 0, 10)),
        "pandan_leaves": float(max(p, 0))
    }
    data = api_post("/experiment", payload, timeout=8)
    return (data or {}).get("message", "").upper() == "PASSED"

def find_threshold_fast(c, e, s, probe_list=(512, 2048, 8192, 16384, 32768), abs_tol=10.0, max_bisect_steps=10):
    try:
        if not experiment_eval(c, e, s, 0.0):
            return 0.0
    except:
        return 0.0
    lo, hi = 0.0, None
    for p in probe_list:
        if experiment_eval(c, e, s, p):
            lo = p
        else:
            hi = p
            break
    if hi is None:
        return float(probe_list[-1])
    steps = 0
    while (hi - lo) > abs_tol and steps < max_bisect_steps:
        steps += 1
        mid = (lo + hi) / 2.0
        if experiment_eval(c, e, s, mid):
            lo = mid
        else:
            hi = mid
    return max(lo, 0.0)

def lhs(n, low=0.0, high=10.0, seed=321):
    random.seed(seed)
    np.random.seed(seed)
    d = 3
    cut = np.linspace(0, 1, n + 1)
    u = np.random.rand(n, d)
    a = cut[:n]
    b = cut[1:n + 1]
    P = u * (b - a)[:, None] + a[:, None]
    for j in range(d):
        np.random.shuffle(P[:, j])
    return low + P * (high - low)

@dataclass
class TrainedModel:
    pipeline: Pipeline
    degree: int
    r2: float

def train_model():
    Xs, Ys = [], []
    need = 24
    print(f"[Train] Collecting {need} samples ...")
    pts = lhs(need, 0, 10, seed=777)
    for i in range(need):
        c, e, s = map(float, pts[i])
        T = find_threshold_fast(c, e, s, abs_tol=10.0)
        Xs.append([c, e, s])
        Ys.append(T)
        if (i + 1) % 4 == 0:
            print(f"  - {i + 1}/{need}, T≈{T:.1f}")
    X = np.array(Xs)
    y = np.array(Ys)
    Xtr, Xval, ytr, yval = train_test_split(X, y, test_size=0.25, random_state=42)
    best = None
    br2 = -1
    bdeg = None
    for deg in (2, 3):
        pipe = Pipeline([
            ("scaler", StandardScaler()),
            ("poly", PolynomialFeatures(degree=deg, include_bias=False)),
            ("reg", Ridge(alpha=1e-6, random_state=42))
        ])
        pipe.fit(Xtr, ytr)
        r2 = r2_score(yval, pipe.predict(Xval))
        print(f"    Degree {deg}: R^2={r2:.6f}")
        if r2 > br2:
            best = pipe
            br2 = r2
            bdeg = deg
    print(f"[Train] Best degree={bdeg}, R^2={br2:.6f}")
    return TrainedModel(best, bdeg, br2), X, y

# ============ Robust /order parsing and retry ============
def parse_ingredient_list(value) -> List[float]:
    if isinstance(value, list):
        return [float(x) for x in value]
    if isinstance(value, str):
        try:
            parsed = json.loads(value)
        except json.JSONDecodeError:
            parsed = ast.literal_eval(value)
        return [float(x) for x in parsed]
    raise ValueError("Unexpected format for ingredient list")

def extract_triples(order_obj) -> List[List[float]]:
    triples = []
    if not order_obj:
        return triples
    # Named keys
    for i in range(1, 4):
        key = f"ingredient_list_{i}"
        if key in order_obj:
            try:
                triples.append(parse_ingredient_list(order_obj[key]))
            except:
                pass
    # Generic values fallback
    if len(triples) < 3:
        for v in order_obj.values():
            if isinstance(v, (list, str)):
                try:
                    li = parse_ingredient_list(v)
                    if len(li) == 3:
                        triples.append(li)
                except:
                    pass
    # Clean to 3 triples
    clean = []
    for t in triples:
        if isinstance(t, list) and len(t) == 3:
            clean.append([float(t[0]), float(t[1]), float(t[2])])
            if len(clean) == 3:
                break
    return clean

def get_order_with_retry(max_fetches: int = 8, short_delay: float = 0.8):
    for attempt in range(1, max_fetches + 1):
        data = api_get("/order", timeout=6)
        order = data.get("order", {})
        token = data.get("token", None)
        triples = extract_triples(order)
        if token and len(triples) == 3:
            return token, triples
        time.sleep(short_delay + random.uniform(0, 0.3))
    raise RuntimeError("Could not fetch 3 ingredient lists with a valid token from /order")

# ============ Taste-test with array-first format + fallback ============
def submit_taste_test(token: str, pandan_values: List[float]):
    # 1) Send as JSON array (what most servers expect)
    payload_array = {"token": token, "result": [float(x) for x in pandan_values]}
    try:
        return api_post("/taste-test", payload_array, timeout=8)
    except requests.HTTPError as e:
        # Fallback: send as JSON string if server insists on string form
        try:
            if e.response is not None:
                print("Array format failed; retrying with string. Server said:", e.response.text)
        except:
            pass
        payload_string = {"token": token, "result": json.dumps([float(x) for x in pandan_values])}
        return api_post("/taste-test", payload_string, timeout=8)

# ============ Orchestration with close-to-threshold margin ============
def main():
    t0 = time.time()
    print("== Limit Theory ML CTF: Auto Solver (fixed taste-test) ==")
    print("[Phase 1] Training ...")
    model, X, y = train_model()
    print(f"[Done] degree={model.degree}, R^2={model.r2:.6f}, samples={len(X)}")
    for i in np.random.choice(len(X), size=min(4, len(X)), replace=False):
        yh = float(model.pipeline.predict(X[i].reshape(1, -1))[0])
        print(f"  sanity: true T={y[i]:.1f}, pred={yh:.1f}, c,e,s={X[i]}")

    print("\n[Phase 2] Fetch order with validation ...")
    token, triples = get_order_with_retry(max_fetches=8, short_delay=0.8)
    print(f"[Prod] Got token and {len(triples)} triples.")
    for i, (c, e, s) in enumerate(triples):
        print(f"  Triple {i}: c={c:.2f}, e={e:.2f}, s={s:.2f}")

    # Small margin to stay within tolerance (close to threshold)
    safety_frac = 0.0002   # 0.3% below threshold
    min_abs_margin = 1.0  # at least 3 units
    preds = []
    for (c, e, s) in triples:
        t_hat = float(model.pipeline.predict(np.array([[c, e, s]]))[0])
        t_hat = max(t_hat, 0.0)
        margin = max(min_abs_margin, safety_frac * t_hat)
        final_value = max(t_hat - margin, 0.0)
        preds.append(round(final_value, 3))
        print(f"  Pred T={t_hat:.2f}  -> submit {final_value:.2f}")

    print(f"[Prod] Submitting taste-test pandan: {preds}")
    resp = submit_taste_test(token, preds)
    print("[Prod] Taste-test response:", resp)
    if "FLAG" in resp:
        print("\n========== FLAG ==========")
        print(resp["FLAG"])
        print("==========================\n")
    else:
        print("No flag in response. Message:", resp)

    print(f"All done in {time.time() - t0:.1f}s")

if __name__ == "__main__":
    main()

== Limit Theory ML CTF: Auto Solver (fixed taste-test) ==
[Phase 1] Training ...
[Train] Collecting 24 samples ...
  - 4/24, T≈1172.0
  - 8/24, T≈19312.0
  - 12/24, T≈8864.0
  - 16/24, T≈10800.0
  - 20/24, T≈1358.0
  - 24/24, T≈9896.0
    Degree 2: R^2=0.999997
    Degree 3: R^2=0.860149
[Train] Best degree=2, R^2=0.999997
[Done] degree=2, R^2=0.999997, samples=24
  sanity: true T=1358.0, pred=1358.2, c,e,s=[1.51834554 0.76468891 4.09016853]
  sanity: true T=10152.0, pred=10150.9, c,e,s=[9.811163   9.3434675  2.59288671]
  sanity: true T=9408.0, pred=9406.8, c,e,s=[6.13589088 2.65552474 6.71468737]
  sanity: true T=3470.0, pred=3471.5, c,e,s=[8.30092988 4.38202381 0.94550211]

[Phase 2] Fetch order with validation ...
[Prod] Got token and 3 triples.
  Triple 0: c=11.45, e=7.64, s=11.42
  Triple 1: c=11.85, e=12.83, s=9.89
  Triple 2: c=7.02, e=6.70, s=11.28
  Pred T=32261.42  -> submit 32254.97
  Pred T=33562.90  -> submit 33556.19
  Pred T=19698.78  -> submit 19694.84
[Prod] Submittin